## 4. Time Series Classification & Prediction Deep Learning
You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is the goal.
File descriptions
* sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
* test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
* sample_submission.csv - a sample submission file in the correct format.
* items.csv - supplemental information about the items/products.
* item_categories.csv - supplemental information about the items categories.
* shops.csv- supplemental information about the shops.

#### Hint:
* Recommended a hybrid CNN-LSTM model 
* Dimensionality Reduction
    - Use Several Classifiers/ Ensemble Method
    - Logistic Regression (with different c values)
    - Random Forest (with different estimator values) 
    - SVM (with different kernels)
    - KNN (with k = 1,2,5,10,20)
    - K (3,5,10) Fold Cross Validation
* Performance Comparison
    - Classification Accuracy, Precision, Recall, Sensitivity, Specificity * AUC, ROC Curve
    - Confusion Matrix

In [1]:
%reload_ext autoreload
%autoreload 2
from Helpers import *
from AccuracyMetrics import *
from CustomMods import *

In [2]:
import gc
gc.collect()

39

In [3]:
# Supress warnings
def warn(*args, **kwargs):
    pass
import warnings
import os as os
import numpy as np
import pandas as pd
import re as regx
import gc as gc
import sys as sys
from datetime import datetime
from time import mktime
import matplotlib
import matplotlib.pyplot as plt

# iPy magic and other cool settings 
%matplotlib inline
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
np.random.seed(42)
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
warnings.warn = warn

from collections import Counter
from scipy.sparse import csr_matrix
from itertools import compress
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [4]:
# Import datasets
item_cats = pd.read_csv('../data/4/item_categories.csv')
items = pd.read_csv('../data/4/items.csv')
sales_train = pd.read_csv('../data/4/sales_train.csv')
test = pd.read_csv('../data/4/test.csv')
shops = pd.read_csv('../data/4/shops.csv')

In [5]:
sales_train.head(5)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [6]:
items = pd.merge(items, item_cats, how='left', on=['item_category_id'])

In [7]:
items.head()

,item_name,item_id,item_category_id,item_category_name
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,Программы - Для дома и офиса (Цифра)
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD
4,***КОРОБКА (СТЕКЛО) D,4,40,Кино - DVD


In [8]:
items.shape

(22170, 4)

In [9]:
features = 25
TfdfVect = TfidfVectorizer(max_features=features)
item_name = pd.DataFrame(TfdfVect.fit_transform(items['item_name']).toarray())

In [10]:
merge_data(items, item_name, 'item_name')

item_name.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.637898,0.0,0.0,...,0.0,0.403761,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.483839
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [11]:
TfdfVect = TfidfVectorizer(max_features=features)
item_category_name = pd.DataFrame(TfdfVect.fit_transform(items['item_category_name']).toarray())
merge_data(items, item_category_name, 'item_cat_name')

In [12]:
TfdfVect = TfidfVectorizer(max_features=features)
shop_name = pd.DataFrame(TfdfVect.fit_transform(shops['shop_name']).toarray())
merge_data(shops, shop_name, 'shop_name')

In [13]:
sales_train = sales_train[(sales_train['item_price'] > 0) & (sales_train['item_cnt_day'] > 0)]

In [15]:
type(sales_train['date'][0])

str

In [16]:
sales_train['date'] = sales_train['date'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y'))
sales_train['year'] = sales_train['date'].apply(lambda x: x.year)
sales_train['month'] = sales_train['date'].apply(lambda x: x.month)

In [39]:
sales_train2 = pd.merge(sales_train_grpby, avg_item_price, how='left', on=['shop_id','item_id','year','month'])
sales_train3 = pd.merge(sales_train2, items, how='left', on=['item_id'])
sales_train4 = pd.merge(sales_train3, shops, how='left', on=['shop_id'])
train = sales_train4

In [46]:
test2 = pd.merge(test, item_price_by_date, how='left', on=['shop_id','item_id'])
test3 = pd.merge(test2, items, how='left', on=['item_id'])
test4 = pd.merge(test3, shops, how='left', on=['shop_id'])
test5 = pd.merge(test4, previous_test_item_cnt[['item_id', 'shop_id', 'item_cnt_prev_month']], how='left', on=['item_id', 'shop_id'])
test6 = pd.merge(test5, test_avg_item_month, how='left', on=['item_id', 'shop_id'])
testdf = test6

In [47]:
for col in ['shop_id', 'item_id', 'item_category_id']:
    train[col] = train[col].astype(str)
    testdf[col] = testdf[col].astype(str)

In [48]:
train.sort_values(by=['year','month'], ascending=[False, False]).head(1)

,shop_id,date_block_num,item_id,year,month,item_cnt_month,item_cnt_prev_month,item_cnt_month_mean,item_price,item_name,...,shop_name15,shop_name16,shop_name17,shop_name18,shop_name19,shop_name20,shop_name21,shop_name22,shop_name23,shop_name24
1576705,58,33,2252,2015,10,4.0,1.0,2.259259,399.0,"Call Of Duty: Modern Warfare 3 [PC, Jewel]",...,0.0,0.0,0.0,0.0,0.0,0.0,0.322815,0.0,0.689588,0.648274


In [49]:
testdf['year'] = 2015
testdf['month'] = 11
testdf['date_block_num'] = 34

In [50]:
train = shuffle(train, random_state=42)

In [51]:
X = train[[col for col in train.columns.values\
           if col not in ['item_name', 'item_category_name', 'shop_name', 'item_cnt_month',\
                          'item_cnt_prev_month', 'item_cnt_month_mean']]].fillna(0)

y = train['item_cnt_month'].fillna(0)

list_training = list(X['date_block_num']<33)
list_testing = list(X['date_block_num']==33)

X_train2 = X[X['date_block_num']<33]
y_train2 = y[list_training].fillna(0)
X_test2 = X[X['date_block_num']==33]
y_test2 = y[list_testing].fillna(0)

In [52]:
reg = ExtraTreesRegressor(n_estimators=25, n_jobs=-1, max_depth=15, random_state=42)
reg.fit(X_train2, y_train2)

y_train_pred = reg.predict(X_train2)
y_test_pred = reg.predict(X_test2)

rmse_train = np.sqrt(mean_squared_error(y_train2, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test2, y_test_pred))

print('rmse_train:', rmse_train)
print('rmse_test:', rmse_test)

rmse_train: 1.9816493854980095
rmse_test: 2.2061106360729092


In [53]:
rf_reg2 = RandomForestRegressor(n_estimators=25, max_depth=10, random_state=42)
rf_reg2.fit(X_train2, y_train2)

y_train2_pred = rf_reg2.predict(X_train2)
y_test2_pred = rf_reg2.predict(X_test2)

In [54]:
rmse_train2 = np.sqrt(mean_squared_error(y_train2, y_train2_pred))
rmse_test2 = np.sqrt(mean_squared_error(y_test2, y_test2_pred))

print('rmse_train:', rmse_train2)
print('rmse_test:', rmse_test2)

rmse_train: 2.126222232885013
rmse_test: 2.2862270245545933


In [56]:
test2df = testdf[[col for col in X_train2.columns.values]].fillna(0)

In [57]:
test_pred = reg.predict(test2df)

In [58]:
test_pred = np.round(test_pred,1)

In [59]:
test['item_cnt_month'] = test_pred

In [60]:
test.to_csv(path_or_buf='final_result.csv', columns=['ID', 'item_cnt_month'], index=False)